# Limpando os dados de uma base

### Passo 1 - Importar a base de dados de clientes de um banco com o resultado do pagamento ou não de um empréstimo

Objetivo: Tratar essa base de origem para separar as "chaves"(letras de números) das colunas da planilha. Criar colunas para categorizar as letras e números separadas. Verficar com o cliente o que essas "chaves" significam. Ex:32FC, Ccinza, Basic-Alpha.	

Case: projeto em um banco cujas as informações não estão bem claras. 
Não se sabe sobre os elementos da planilha(o que são) - Como: 32FC, CCinza, Basic-Alpha. 

Apenas sabe-se que na coluna pagamento, o número 1 indica que o cliente pagou, e 0 que ele não pagou o empréstimo.


In [1]:
# Importando biblioteca

import pandas as pd

In [2]:
# Lendo o arquivo

base = pd.read_excel('ChavesClientes.xlsx', sheet_name = 'base')

# Mostrar as últimas 5 linhas da tabela

base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


### Passo 2 - Verificar a cardinalidade desses dados

In [3]:
# Agrupar ChaveSituacao com Pagamento com groupby - para verificar se foi pago ou não
# contar para cada chavesituacao quando foram pagos ou não - count()
# EXEMPLO: Na ChaveSituacao 26FD, 2 não foram pagos - Número 0 à esquerda na coluna Pagamento
# A partir da 25FD mostra os que foram pagos. Número 1 à esquerda na coluna Pagamento


base.groupby(['Pagamento','ChaveSituacao'])['Pagamento'].count()


Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### Passo 3 - Questionar ao cliente quando essa base - Anotações sobre o que foi passado pelo cliente


#### Ao mapear com o responsável pela área, ele informou o que representa cada coluna:
- ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
- ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa - probabilidade do cliente sair da empresa
- CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
- Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta


#### EXEMPLO: 

28FC 
- 28 anos
- Feminino
- Casada

Pagamento - 0 (não pagou)
Quantidade de 28FC que não pagaram - 2 pessoas


25MV  
- 25 aos
- Masculino
- Viúvo

Pagamento - 1 ( pagou)
Quantidade de 25MV que pagaram - 1 pessoa

### Passo 4 - Separando dados por colunas de acordo com as características informadas pelo cliente

#### Tratando a coluna ChaveSituacao

In [4]:
# Tratar separando a string da coluna
        
texto = '32FC'


In [5]:
# texto -1, imprime o C
# se fosse - 2, imprime o F
# se fosse [2:], imprime os dois últimos -  FC
# se fosse [:2], imprime os dois primeiros - 32
# se fosse [2:3], imprime apenas o F - começa em 2 e vai até a posição 3, porém o 3 não conta
# se fosse [3:4], imprime apenas o C - começa em 3 e vai até a posição 4, porém o 4 não conta

texto = [-1]

texto


[-1]

In [6]:
# Inserindo a coluna Idade - separando a idade da string(Ex:FC)
# str[:2], mostra os dois primeiros - idade
# str[2:3], mostra apenas o gênero - genero
# str[-1], mostra o último elemento - estado civil


base['Idade'] = base.ChaveSituacao.str[:2]

base['Gênero'] = base.ChaveSituacao.str[2:3]

base['EstadoCivil'] = base.ChaveSituacao.str[-1]


# mostra apenas as últimas 5 linhas da tabela

base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Gênero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


In [7]:
# Mesmo agrupamento que anterior ,porém por estado civil

base.groupby(['Pagamento','EstadoCivil'])['Pagamento'].count()


Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

#### EXEMPLO: 

Pagamento de acordo com o estado civil 

0 - não pagaram 

- Casado(C) - 3 pessoas
- Divorciado(D) - 4 pessoas

1 - pagaram

- Casado(C) - 4 pessoas
- Viúvo(V) - 5 pessoas - todos eles pagaram, talve para essa empresa a indicação de viúvo seja uma coisa boa

#### Tratando a coluna CatCliente

In [8]:
# Verificando todos os valores

display(base)

# PARA SEPARAR AS CATEGORIAS -

# se pegar a str[5:], de 5 últimos, poderia saber se é alpha, black, basic(Têm 5 letras). Porém,

# o Beta tem apenas 4 letras e,

# a categoria Platinum tem mais letras, mais que 5 (tem 8). Não é possível seguir essa lógica,

# porque não printaria a categoria platinum inteira. Apenas (tinum), se printar os 5 últimos

# dessa forma, não se pode usar a lógica anteriord de str[].

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Gênero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D
7,8,31MD,C-Cinza,Basic,0,31,M,D
8,9,28FS,A-Cinza,Black,1,28,F,S
9,10,31MV,C+Amarelo,Platinum,1,31,M,V


In [9]:
# Fazer o Split(divisão) de um valor, baseado em um delimitador

texto = 'Basic-Alpha'

In [10]:
# sepapando basic do alpha do traço ('-')
# se fosse ('A') -> ['Basic ', 'lpha']
# se fosse texto = 'Basic Alpha' sem o traço, colocando ('-') -> ['Basic Alpha']

texto.split('-')

['Basic', 'Alpha']

In [11]:
# Aplicando o split para criar uma nova coluna chamada Categoria

# base['Categoria'] = base.CatCliente.split('-')  -> dá erro porque está tentando fazer um split em uma series, e não consegue

# Transformar numa string para não dar erro - str.split('-')

base['Categoria'] = base.CatCliente.str.split('-')


In [12]:
# Fazer apenas nas últimas 5 linhas

base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Gênero,EstadoCivil,Categoria
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,"[Basic, Alpha]"
1,2,25MV,AAmarelo,Black,1,25,M,V,[Black]
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,"[Basic, Beta]"
3,4,26FD,BAmarelo,Black,0,26,F,D,[Black]
4,5,26FD,C-Amarelo,Black,0,26,F,D,[Black]


In [13]:
# Porém, é mais vantajoso mostrar apenas uma categoria como Basic, platinum, etc - utilizar .str.get(0)
# onde irá pegar o primeiro elemento (0)

base['Categoria'] = base.CatCliente.str.split('-').str.get(0)

# Criando uma categoria VIP  - str.get(1) - irá pegar o segundo elemento(1), no qual indica a categ. VIP ou não
# o que não tiver categoria VIP, virá como NaN (valor vazio)

base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)


base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Gênero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


#### Tratando a coluna ClassRisco

In [14]:
# Separar 

# Existe um padrão totalmente diferente na coluna Class risco
# como Ccinza, B-Amarelo - letras, traços (+, -)
#Classificação do cliente como (A,B,C) e indicador (+,- ou vazio).Cores de acordo com a probabilidade do cliente sair da empresa

# Caso não consiga separar pela posição do texto ou por um delimitador, tem-se a opção de usar Regex
# importar biblioteca re - regex

In [15]:
# Importando regex

import re

- Regex:
    - ^: Começa com
    - $: Termina com
    - *: O último caracter repetido 0 ou mais vezes
    - +: O último caracter repetido 1 ou mais vezes
    - ?: O último caracter repetido 0 ou 1 vez
    - [A-Z]: qualquer valor em maiúsculo

In [16]:
# Aplicando regex para separar B+ , C-, etc

# aplicar a fórmula regex na coluna ClassRisco-> apply
# x -> elementos da coluna ClassRisco - ao inserir o X, substitui-se os elementos por x.Ele fará esse código para toda a coluna.
# x-> findall

#lambda -> para fazer essa função de x: refindall()


# re.findall -> ^ começa com A-Z maiúsculo 
# ^A-Z - não quer um A-Z, quer-se o próximo elemento, desde que ele não seja maíusculo

#ERRO NA BASE -> CCinza, por isso, ficaria [Cc] na linha, e não [C]. Então, usar
# a-z -> letras minúsculas


# ? é o que tem depois da letra(+,-) Ex: B + , C -
# [0] - tirar os colchetes das letras(primeiro valor). Ex: na tabela ficaria [B+] 

base['Risco'] = base.ClassRisco.apply(lambda x: re.findall("^[A-Z][^A-Za-z]?", x)[0])

In [17]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Gênero,EstadoCivil,Categoria,CatVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


### Passo 5 - Verificar informações da planilha (info)

In [18]:
# info()

base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Gênero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


In [19]:
# a idade está como um objeto, deveria está como int, pois é um número

#### Transformando a coluna "Idade" em númerico

In [20]:
# pd.to_numeric - função do pandas para converter a idade para número - int

base['Idade'] = pd.to_numeric(base['Idade'])

# fazer info() novamente para verificar

In [21]:
# verificando
base.info()

# a idade foi transformada para int - inteiro

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Gênero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


In [22]:
# todas as outras colunas têm 20 valores não nulos (não tem vazios)
# o CatVIP tem 8 valores não nulos, ou seja 20 - 8 = 12 dos valores estão vazios. 

#### Tratando os 12 valores vazios da coluna "CatVIP"

In [23]:
# localizando os valores vazios - base.loc
# base.CatVIP.isnull() - para valres vazios - NaN
# filtrar a coluna CatVIP , igualar a comum para não ter valores vazios

# todos os que estiverem NaN serão trocados por "Comum"

base.loc[base.CatVIP.isnull(), 'CatVIP'] = 'Comum'

In [24]:
# verificar info() novamente

In [25]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Gênero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         20 non-null     object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


In [26]:
# todos os 12 valores vazios de CatVIP foram substituídos por "Comum". Resultando em 20 não nulos - non-null

In [27]:
# com isso foi feita uma boa limpeza de dados para se trabalhar com a tabela

### Passo 6 - Verificar o tratamento de dados feitos anteriormente

In [28]:
# verificando a categoria

base.groupby(['Pagamento','Categoria'])['Pagamento'].count()


Pagamento  Categoria
0          Basic        3
           Black        3
           Platinum     1
1          Basic        3
           Black        4
           Platinum     6
Name: Pagamento, dtype: int64

In [29]:
# verifica-se que o cliente Platinum paga melhor - 6, maior quantidade

#o cliente Basic e Black estão praticamente na mesma categoria, pois o 0(na coluna pagamento) indica que não foi pago. 3 em ambos 

# 0          Basic        3
#            Black        

# O valor 1 indica que o pagamento foi feito. 
# 1          Basic        3
#            Black        4

In [30]:
base.groupby(['Pagamento','CatVIP'])['Pagamento'].count()


Pagamento  CatVIP
0          Alpha     1
           Beta      2
           Comum     4
1          Alpha     3
           Beta      2
           Comum     8
Name: Pagamento, dtype: int64

In [31]:
# Verificando-se o cliente Alpha, apenas 1 não pagou. 0 na coluna "pagamento" indica que não foi pago

# 0          Alpha     1

# Porém, observa-se que a maioria cumpriu com o seu pagamento(3 pagaram). O 1 na coluna "pagamento" indica que foi pago.

# 1          Alpha     3


# Já o cliente "Comum" que não é VIP, está melhor pois 8 pagaram e 4 não pagaram. 

# Alpha e Beta são clientes VIPs

In [32]:
base.groupby(['Pagamento','Risco'])['Pagamento'].count()


Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64

In [33]:
# Ao verificar a coluna "Risco", todos os clientes A (A+ e A-) pagaram. Número 1 na coluna "pagamento" indica que foi pago

# Em relação ao cliente C-, nenhum deles pagaram (5) . O número 0 na coluna "pagamento" indica que não foi pago. 

# pois não há C- na coluna de pagamento 1, apenas no 0. Porém, o C+ pagaram